# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("education_middle_highschool_data.txt") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 162, which is longer than the specified 100
Created a chunk of size 200, which is longer than the specified 100
Created a chunk of size 183, which is longer than the specified 100
Created a chunk of size 204, which is longer than the specified 100
Created a chunk of size 210, which is longer than the specified 100
Created a chunk of size 167, which is longer than the specified 100
Created a chunk of size 189, which is longer than the specified 100
Created a chunk of size 185, which is longer than the specified 100
Created a chunk of size 169, which is longer than the specified 100
Created a chunk of size 191, which is longer than the specified 100
Created a chunk of size 218, which is longer than the specified 100
Created a chunk of size 184, which is longer than the specified 100
Created a chunk of size 183, which is longer than the specified 100
Created a chunk of size 174, which is longer than the specified 100
Created a chunk of size 197, which is longer tha

In [5]:
docs[0]

Document(page_content='如何提高孩子的学习兴趣？\n\n[家长问题] 我的孩子对学习没有兴趣，应该怎么办？\n[专家回答] 提高学习兴趣可以从寻找孩子的兴趣点开始，然后将这些兴趣点与学科内容相结合。同时，家长的积极参与和鼓励也非常重要。可以设定短期和长期的目标，让孩子感受到学习的成就感。此外，参加一些与学习内容相关的活动或比赛，也能增加学习的乐趣。')

In [6]:
len(docs)

35

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "孩子不爱学习怎么办？"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

孩子厌学怎么办？

[家长问题] 最近孩子对学校表现出明显的厌学情绪，我该怎么办？
[专家回答] 面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。

如何提高孩子的学习兴趣？

[家长问题] 我的孩子对学习没有兴趣，应该怎么办？
[专家回答] 提高学习兴趣可以从寻找孩子的兴趣点开始，然后将这些兴趣点与学科内容相结合。同时，家长的积极参与和鼓励也非常重要。可以设定短期和长期的目标，让孩子感受到学习的成就感。此外，参加一些与学习内容相关的活动或比赛，也能增加学习的乐趣。

孩子对特定学科不感兴趣，怎样激发他们的兴趣？

[家长问题] 我的孩子对数学一点都不感兴趣，这怎么能激发他对数学的兴趣？
[专家回答] 激发孩子对特定学科的兴趣，可以尝试将学科内容与孩子的兴趣爱好相结合，寻找实际生活中与数学相关的有趣应用，如游戏、体育统计或者艺术设计等。同时，可以寻找寓教于乐的教学资源，比如数学游戏、视频或者互动软件，使学习变得更加生动有趣。鼓励孩子参与数学竞赛或项目，以实践和挑战的方式增加他们的兴趣。

孩子总是抱怨学校生活无聊，我该怎么办？

[家长问题] 孩子对学校生活感到厌倦，总是说无聊，如何激发他的兴趣？
[专家回答] 对抗学校生活的乏味感，可以鼓励孩子参与学校的课外活动或社团，这不仅能够拓展他们的兴趣爱好，还能帮助他们建立友谊和社交技能。与孩子一起探索他们的兴趣，看看是否可以在学校找到相关的活动或课程。同时，鼓励孩子设立个人目标，如学习新技能或提高成绩，以增加学校生活的目的感。



In [11]:
db.save_local("education_middle_high_school")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x744ffea0a1a0>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

孩子厌学怎么办？

[家长问题] 最近孩子对学校表现出明显的厌学情绪，我该怎么办？
[专家回答] 面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。

如何提高孩子的学习兴趣？

[家长问题] 我的孩子对学习没有兴趣，应该怎么办？
[专家回答] 提高学习兴趣可以从寻找孩子的兴趣点开始，然后将这些兴趣点与学科内容相结合。同时，家长的积极参与和鼓励也非常重要。可以设定短期和长期的目标，让孩子感受到学习的成就感。此外，参加一些与学习内容相关的活动或比赛，也能增加学习的乐趣。

孩子对特定学科不感兴趣，怎样激发他们的兴趣？

[家长问题] 我的孩子对数学一点都不感兴趣，这怎么能激发他对数学的兴趣？
[专家回答] 激发孩子对特定学科的兴趣，可以尝试将学科内容与孩子的兴趣爱好相结合，寻找实际生活中与数学相关的有趣应用，如游戏、体育统计或者艺术设计等。同时，可以寻找寓教于乐的教学资源，比如数学游戏、视频或者互动软件，使学习变得更加生动有趣。鼓励孩子参与数学竞赛或项目，以实践和挑战的方式增加他们的兴趣。



In [15]:
docs = topK_retriever.get_relevant_documents("Where your favorite song?")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

孩子对特定学科不感兴趣，怎样激发他们的兴趣？

[家长问题] 我的孩子对数学一点都不感兴趣，这怎么能激发他对数学的兴趣？
[专家回答] 激发孩子对特定学科的兴趣，可以尝试将学科内容与孩子的兴趣爱好相结合，寻找实际生活中与数学相关的有趣应用，如游戏、体育统计或者艺术设计等。同时，可以寻找寓教于乐的教学资源，比如数学游戏、视频或者互动软件，使学习变得更加生动有趣。鼓励孩子参与数学竞赛或项目，以实践和挑战的方式增加他们的兴趣。

如何提高孩子的学习兴趣？

[家长问题] 我的孩子对学习没有兴趣，应该怎么办？
[专家回答] 提高学习兴趣可以从寻找孩子的兴趣点开始，然后将这些兴趣点与学科内容相结合。同时，家长的积极参与和鼓励也非常重要。可以设定短期和长期的目标，让孩子感受到学习的成就感。此外，参加一些与学习内容相关的活动或比赛，也能增加学习的乐趣。

孩子总是抱怨学校生活无聊，我该怎么办？

[家长问题] 孩子对学校生活感到厌倦，总是说无聊，如何激发他的兴趣？
[专家回答] 对抗学校生活的乏味感，可以鼓励孩子参与学校的课外活动或社团，这不仅能够拓展他们的兴趣爱好，还能帮助他们建立友谊和社交技能。与孩子一起探索他们的兴趣，看看是否可以在学校找到相关的活动或课程。同时，鼓励孩子设立个人目标，如学习新技能或提高成绩，以增加学校生活的目的感。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

孩子厌学怎么办？

[家长问题] 最近孩子对学校表现出明显的厌学情绪，我该怎么办？
[专家回答] 面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。

如何提高孩子的学习兴趣？

[家长问题] 我的孩子对学习没有兴趣，应该怎么办？
[专家回答] 提高学习兴趣可以从寻找孩子的兴趣点开始，然后将这些兴趣点与学科内容相结合。同时，家长的积极参与和鼓励也非常重要。可以设定短期和长期的目标，让孩子感受到学习的成就感。此外，参加一些与学习内容相关的活动或比赛，也能增加学习的乐趣。

孩子对特定学科不感兴趣，怎样激发他们的兴趣？

[家长问题] 我的孩子对数学一点都不感兴趣，这怎么能激发他对数学的兴趣？
[专家回答] 激发孩子对特定学科的兴趣，可以尝试将学科内容与孩子的兴趣爱好相结合，寻找实际生活中与数学相关的有趣应用，如游戏、体育统计或者艺术设计等。同时，可以寻找寓教于乐的教学资源，比如数学游戏、视频或者互动软件，使学习变得更加生动有趣。鼓励孩子参与数学竞赛或项目，以实践和挑战的方式增加他们的兴趣。

孩子总是抱怨学校生活无聊，我该怎么办？

[家长问题] 孩子对学校生活感到厌倦，总是说无聊，如何激发他的兴趣？
[专家回答] 对抗学校生活的乏味感，可以鼓励孩子参与学校的课外活动或社团，这不仅能够拓展他们的兴趣爱好，还能帮助他们建立友谊和社交技能。与孩子一起探索他们的兴趣，看看是否可以在学校找到相关的活动或课程。同时，鼓励孩子设立个人目标，如学习新技能或提高成绩，以增加学校生活的目的感。



### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'孩子厌学怎么办？\n\n[家长问题] 最近孩子对学校表现出明显的厌学情绪，我该怎么办？\n[专家回答] 面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。'

In [21]:
docs[0].page_content.split("[专家回答] ")

['孩子厌学怎么办？\n\n[家长问题] 最近孩子对学校表现出明显的厌学情绪，我该怎么办？\n',
 '面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。']

In [22]:
ans = docs[0].page_content.split("[专家回答] ")[-1]

In [23]:
ans

'面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[专家回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "孩子上学被欺负了怎么办？"

print(sales(query))

['面对校园霸凌，首要任务是确保孩子的安全和情绪支持。与孩子进行开放且支持性的对话，鼓励他们分享自己的感受。了解具体情况后，应及时与学校沟通，寻求学校的协助处理此事。同时，教育孩子关于自尊和边界的重要性，以及如何安全地应对潜在的霸凌行为。在必要时，寻求专业心理咨询师的帮助。']


In [26]:
print(sales(query, 0.75))

['面对校园霸凌，首要任务是确保孩子的安全和情绪支持。与孩子进行开放且支持性的对话，鼓励他们分享自己的感受。了解具体情况后，应及时与学校沟通，寻求学校的协助处理此事。同时，教育孩子关于自尊和边界的重要性，以及如何安全地应对潜在的霸凌行为。在必要时，寻求专业心理咨询师的帮助。', '面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。', '对抗学校生活的乏味感，可以鼓励孩子参与学校的课外活动或社团，这不仅能够拓展他们的兴趣爱好，还能帮助他们建立友谊和社交技能。与孩子一起探索他们的兴趣，看看是否可以在学校找到相关的活动或课程。同时，鼓励孩子设立个人目标，如学习新技能或提高成绩，以增加学校生活的目的感。', '处理孩子与老师之间的矛盾首先需要了解矛盾的具体情况。与孩子进行开放式的对话，了解孩子的感受和具体事情。同时，也可以适当与老师沟通，尝试从老师的角度理解情况。在这个过程中，重要的是保持中立，既不盲目站在孩子一边，也不完全否定孩子的感受。引导孩子学习如何从不同角度看问题，并教会他们适当的解决冲突的方法。']


In [27]:
query = "买学区房那么贵，值得吗？"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。', '孩子在不同环境下的表现差异可能是因为社交压力、适应环境的不同方式或是期望值的差异。与孩子进行开放和支持性的对话，了解他们在学校的感受和经历。与学校老师保持沟通，了解孩子在学校的行为和表现。提供一个安全和支持的环境，鼓励孩子表达自己的感受和担忧，共同探索解决方式。', '对抗学校生活的乏味感，可以鼓励孩子参与学校的课外活动或社团，这不仅能够拓展他们的兴趣爱好，还能帮助他们建立友谊和社交技能。与孩子一起探索他们的兴趣，看看是否可以在学校找到相关的活动或课程。同时，鼓励孩子设立个人目标，如学习新技能或提高成绩，以增加学校生活的目的感。', '面对高中生活的焦虑，重要的是与孩子进行开放和支持性的对话，了解他们的具体担忧。鼓励他们表达自己的感受，并提供安慰和理解。可以一起探讨和制定应对策略，比如参加学校组织的新生活动，提前熟悉校园环境，或是与未来的同学建立联系。提醒孩子每个人都会经历适应期，而且他们不是孤单一人。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [29]:
qa_chain({"query": "为什么一定要买学区房？"})

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '为什么一定要买学区房？',
 'result': '买学区房主要是因为在很多国家和地区，公立学校的入学往往与学生的居住地挂钩。学区房指的是那些位于优质公立学校（尤其是小学和中学）划定的招生范围内的住宅。以下是几个主要原因，为什么人们会选择买学区房：\n\n1. 教育资源：优质的学区通常意味着那里的学校有着更好的教育资源，包括有资历的教师、良好的学术氛围、丰富的课外活动等。\n\n2. 孩子教育：家长通常希望给孩子最好的教育，而入读一个好学校是很多家长认为可以提供高质量教育的方式。\n\n3. 房产价值：学区房通常保值和升值的潜力较大。即使家长不考虑孩子教育的因素，作为一种投资，学区房也往往是一个不错的选择。\n\n4. 社区环境：优质学区的周边环境通常也比较好，治安、绿化、公共设施等方面可能都更有保障。\n\n5. 稳定性：一旦购买了学区房，如果政策不变，家长可以相对放心孩子的教育问题，减少了因搬家等原因导致的学校变动。\n\n然而，学区房也有其不足之处，比如价格通常较高，可能会给家庭经济带来压力。此外，过分追求学区房也可能导致资源分配不均，加剧社会分层。因此，是否购买学区房需要根据个人的实际情况和价值判断来决定。'}

In [30]:
qa_chain({"query": "孩子不好好学习怎么办？"})

{'query': '孩子不好好学习怎么办？',
 'result': '面对孩子不好好学习的问题，可以采取以下几个策略：\n\n1. 了解原因：首先与孩子进行耐心的对话，了解他们不愿意学习的具体原因，可能是学习压力、人际关系问题、对学习内容的不感兴趣等。\n\n2. 调整学习计划：根据孩子的实际情况，调整学习计划，安排合理的学习时间和休息时间，减少学习压力。\n\n3. 寻求帮助：如果问题复杂，可以求助于学校老师或专业的心理辅导老师，共同寻找解决方案。\n\n4. 提高学习兴趣：尝试将孩子的兴趣与学习内容结合，设定短期和长期目标，让孩子感受到学习的成就感。\n\n5. 家长支持：家长应该提供一个积极的家庭环境，给予孩子足够的支持和鼓励，避免因成绩问题对孩子产生负面影响。\n\n6. 鼓励课外活动：鼓励孩子参加自己感兴趣的课外活动，这不仅能增加学习以外的乐趣，也有助于培养孩子的综合能力。\n\n记得在处理这些问题时保持耐心和理解，避免过于严厉的批评，这样可以帮助孩子重建对学习的积极态度。'}

In [31]:
print(sales("孩子不好好学习怎么办？"))

['面对孩子的厌学情绪，重要的是找出背后的原因。与孩子进行耐心的对话，了解他们的感受和遇到的困难。可能的原因包括学习压力、人际关系问题或对学习内容的不感兴趣。根据具体情况，考虑调整学习计划，减少压力，或是寻求学校老师和心理辅导老师的帮助。同时，鼓励孩子参加自己感兴趣的课外活动，增加学校生活的乐趣。', '首先，与孩子进行开放和支持性的对话，尝试了解成绩下滑的原因，是否是学习方法不当、学习压力太大，或是其他生活方面的干扰。根据问题的具体情况，制定相应的对策，比如调整学习计划、提供额外的辅导或是改善生活习惯。同时，重要的是保持耐心，给予孩子足够的支持和鼓励，避免施加过多压力。', '提高学习兴趣可以从寻找孩子的兴趣点开始，然后将这些兴趣点与学科内容相结合。同时，家长的积极参与和鼓励也非常重要。可以设定短期和长期的目标，让孩子感受到学习的成就感。此外，参加一些与学习内容相关的活动或比赛，也能增加学习的乐趣。', '提高数学成绩首先需要弄清楚孩子在数学学习中遇到的具体问题。可以和孩子一起复习错题，找出错误的原因，是概念不理解还是计算错误。鼓励孩子多做练习，特别是基础题型的训练，以加强对基本概念的理解和运用。如果需要，也可以考虑请家教或参加补习班。']


## 加载 FAISS 向量数据库已有结果

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("education_middle_high_school", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [33]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
qa_chain({"query": "我希望我的孩子和班上最好的学生坐在一个桌，这样好吗？"})

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我希望我的孩子和班上最好的学生坐在一个桌，这样好吗？',
 'result': '这取决于具体的情况和学校的规定。将你的孩子和班上最好的学生坐在一起可能会有一些好处，比如他们可以互相学习和激励。然而，这也可能会给你的孩子带来压力，或者让他们感到自己需要和其他学生竞争。最好的做法可能是和你的孩子以及他们的老师讨论这个问题，看看这样做是否最符合你的孩子的需求和利益。'}

In [35]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [36]:
qa_chain({"query": "我希望我的孩子和班上最好的学生坐在一个桌，这样好吗？"})

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我希望我的孩子和班上最好的学生坐在一个桌，这样好吗？',
 'result': '这主要取决于学校的规定和教师的安排。一般来说，如果这种安排不会影响到课堂秩序和其他学生的学习，那么这是可以的。然而，也要考虑到孩子的感受，如果他们感到压力过大或者不舒服，这可能不是最好的选择。最好和孩子的老师讨论这个问题，看看他们的意见。'}

In [37]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [38]:
result = qa_chain({"query": "我希望我的孩子和班上最好的学生坐在一个桌，这样好吗？"})

/home/parallels/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [39]:
result

{'query': '我希望我的孩子和班上最好的学生坐在一个桌，这样好吗？',
 'result': '这完全取决于您的个人观点和学校的规定。有些人认为，这样可以激发孩子的学习动力，从优秀的学生那里学习好的习惯。但也有可能产生压力，让孩子感到不舒服。建议您和孩子以及他们的老师讨论这个问题，看看这是否是一个好主意。',
 'source_documents': []}

In [40]:
result['result']

'这完全取决于您的个人观点和学校的规定。有些人认为，这样可以激发孩子的学习动力，从优秀的学生那里学习好的习惯。但也有可能产生压力，让孩子感到不舒服。建议您和孩子以及他们的老师讨论这个问题，看看这是否是一个好主意。'